In [2]:
!pip install -Uqq fastbook ipywidgets nbdev

import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)
# your drive home will be at /content/gdrive/MyDrive/
# your notebooks might be at /content/gdrive/MyDrive/Colab Notebooks/
BASE_DIR = "/content/gdrive/MyDrive/Colab Notebooks/fastai_book"
# BASE_DIR = "./work"

# https://wandb.ai/wandb/common-ml-errors/reports/How-To-Use-GPU-with-PyTorch---VmlldzozMzAxMDk
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

path = untar_data(URLs.MNIST_SAMPLE, data=f"{BASE_DIR}/data")

def filesToStackedTensors(d1, d2):
  t = [tensor(Image.open(f)).to(device) for f in (path/d1/d2).ls().sorted()]
  return torch.stack(t).float()/255

def x_y(stacked_x, stacked_y):
  x = torch.cat([stacked_x, stacked_y]).view(-1, 28*28)
  y = tensor([1]*len(stacked_x) + [0]*len(stacked_y)).unsqueeze(1).to(device)
  return (x, y)

train_x, train_y = x_y(filesToStackedTensors('train', '3'), \
                       filesToStackedTensors('train', '7'))
valid_x, valid_y = x_y(filesToStackedTensors('valid', '3'), \
                       filesToStackedTensors('valid', '7'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# how does multiplication work with tensors
a1 = tensor([1, 2, 3])
b1 = tensor([4, 4, 4])
print(f'b.T: {b1.T}, a1 * b1.T: {a1 * b1.T}')
b2 = tensor([[4],[4],[4]])
print(f'b.T: {b2.T}, a1 * b2.T: {a1 * b2.T}')

a2 = tensor([[1,2,3]])
a3 = tensor([[[1,2,3]]])
print(f'a2 * b2.T: {a2 * b2.T}')
print(f'a3 * b2.T: {a3 * b2.T}')

# can't multiply if sizes don't match
# b3 = tensor([1,2,3,4])
# a1 * b3, a1 * b3.T

In [58]:
def init_params(size, std=1.0): return (torch.randn(size) * std).to(device).requires_grad_()
weights = init_params((28*28, 1))
print(f'weights.shape: {weights.shape}')
print(f'weights[0:3]: {weights[0:3]}')
bias = init_params(1)
print(f'bias: {bias}')

# x = train_x[0].unsqueeze(0)
print(f'train_x len: {len(train_x)}')
print(f'train_x shape: {train_x.shape}')
print(f'train_x[0] shape: {train_x[0].shape}')
print(f'train_x[0][0:3] shape: {train_x[0][0:3]}')
prediction = (train_x[0]*weights.T).sum() + bias
print(f'prediction for first image: {prediction}')
prediction = (train_x[0]*(weights * 1.0001).T).sum() + bias
print(f'prediction for first image: {prediction}')
prediction = (train_x[0]*(weights*0.9999).T).sum() + bias
print(f'prediction for first image: {prediction}')

weights.shape: torch.Size([784, 1])
weights[0:3]: tensor([[ 1.4049],
        [-0.2031],
        [-0.9893]], grad_fn=<SliceBackward0>)
bias: tensor([1.5683], requires_grad=True)
train_x len: 12396
train_x shape: torch.Size([12396, 784])
train_x[0] shape: torch.Size([784])
train_x[0][0:3] shape: tensor([0., 0., 0.])
prediction for first image: tensor([10.6491], grad_fn=<AddBackward0>)
prediction for first image: tensor([10.6500], grad_fn=<AddBackward0>)
prediction for first image: tensor([10.6482], grad_fn=<AddBackward0>)


In [44]:
import time

def linear1(x, w, b): return x@w + b
s1 = time.time()
preds1 = linear1(train_x, weights, bias)
print(f'time taken using linear1: {time.time() - s1}')
print(f'preds1.shape: {preds1.shape}')

def linear2(x, w, b):
  preds = torch.zeros((len(x), 1))
  for i in range(len(x)):
    preds[i] = ((x[i]*w.T).sum() + bias)
  return preds

# s2 = time.time()
# preds2 = linear2(train_x, weights, bias)
# print(f'time taken using linear2: {time.time() - s2}')
# print(f'preds2.shape: {preds2.shape}')

a1 = ((preds1>0).float() == train_y).float().mean()
# a2 = ((preds2>0).float() == train_y).float().mean()
print(f'a1: {a1}')

def loss(preds, y):
  s = preds.sigmoid()
  return torch.where((y == 1), 1-s, s).float().mean()

loss(preds1, train_y)

time taken using linear1: 0.011233806610107422
preds1.shape: torch.Size([12396, 1])


tensor(0.5128)

In [45]:
# what is the effect of making small changes to weights on the predictions

preds3 = linear1(train_x, weights*1.2, bias)
a3 = ((preds3>0).float() == train_y).float().mean()
preds4 = linear1(train_x, weights*0.8, bias)
a4 = ((preds4>0).float() == train_y).float().mean()
a1, a3, a4

(tensor(0.5128), tensor(0.5142), tensor(0.5119))

In [66]:
# gradient descent that works on entire data set per epoch

def gradient_descent(x, y, w, b):
  l = 0
  lr = 1
  for i in range(100):
    p = linear1(x, w, b)
    l = loss(p, y)
    l.backward()
    w.data = w.data - w.grad * lr
    b.data = b.data - b.grad * lr
    w.grad = b.grad = None

  return l, w, b

l, w, b = gradient_descent(train_x, train_y, weights, bias)
print(f'final loss: {l}')

final loss: 0.03422604501247406


In [69]:
# analyse accuracy using the above function

a = ((linear1(train_x, w, b) > 0.5).float() == train_y).float().mean()
print(f'training set accuracy: {a}')

a = ((linear1(valid_x, w, b) > 0.5).float() == valid_y).float().mean()
print(f'validation set accuracy: {a}')

accuracy: 0.968860924243927
accuracy: 0.9622178673744202


In [55]:
# How Dataloader works
coll = range(15)
dl = DataLoader(coll, batch_size=5, shuffle=True)
print(f'list(dl): {list(dl)}')
for x in dl:
  print(f'len: {len(dl)}')

list(dl): [tensor([14,  9,  7,  0, 12]), tensor([ 6, 11,  4,  2,  8]), tensor([ 5,  3,  1, 13, 10])]
len: 3
len: 3
len: 3


In [ ]:
#??zip
dset = list(zip(train_x, train_y))
x, y = dset[0]
x.shape, y
len(dset), type(dset), type(dset[0]), type(dset[0][0]), len(dset[0][0]), type(dset[0][1])
valid_dset = list(zip(valid_x, valid_y))

In [60]:
# gradient descent that works on a batch of data per epoch

def gradient_descent_batches(dl, weights, bias):
  l = 0
  lr = 1
  for x, y in dl:
    p = linear1(x, weights, bias)
    l = loss(p, y)
    l.backward()
    weights.data = weights.data - weights.grad.data
    bias.data = bias.data - bias.grad.data
    weights.grad = bias.grad = None
  return l, weights, bias

dl = DataLoader(dset, batch_size=255, shuffle=True)
valid_dl = DataLoader(valid_dset, batch_size=255, shuffle=True)

gradient_descent_batches(dl, weights, bias)

tensor(0.0599, grad_fn=<MeanBackward0>)